In [1]:
!pip install striprtf

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from striprtf.striprtf import rtf_to_text
import json
import pandas as pd
import warnings

In [3]:
file = open("algoparams_from_ui.json.rtf","r")
example = file.read()
text = rtf_to_text(example)
file.close()

In [4]:
data = json.loads(text)

In [5]:
df = pd.read_csv("iris.csv")
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [6]:
#1 read target and type of prediction
target_var = data["design_state_data"]["target"]["target"]
prediction_type = data["design_state_data"]["target"]["prediction_type"]

In [7]:
#features imputation details

design = data["design_state_data"]["feature_handling"]
sepal_length_imp_type = design["sepal_length"]["feature_details"]["impute_with"]
sepal_length_imp_val = design["sepal_length"]["feature_details"]["impute_value"]

sepal_width_imp_type = design["sepal_width"]["feature_details"]["impute_with"]
sepal_width_imp_val = design["sepal_width"]["feature_details"]["impute_value"]

petal_length_imp_type = design["petal_length"]["feature_details"]["impute_with"]
petal_length_imp_val =design["petal_length"]["feature_details"]["impute_value"]

petal_width_imp_type = design["petal_width"]["feature_details"]["impute_with"]
petal_width_imp_val = design["petal_width"]["feature_details"]["impute_value"]


from sklearn.impute import SimpleImputer
if sepal_length_imp_type == 'Average of values':
  imputer1 = SimpleImputer(strategy = 'mean')
  df['sepal_length']= imputer1.fit_transform(df[['sepal_length']])
if sepal_length_imp_type == 'custom':
  imputer1 = SimpleImputer(strategy = 'constant',fill_value = sepal_length_imp_val)
  df['sepal_length']= imputer1.fit_transform(df[['sepal_length']])
if sepal_length_imp_type == 'median':
  imputer1 = SimpleImputer(strategy = 'median')
  df['sepal_length']= imputer1.fit_transform(df[['sepal_length']])




if sepal_width_imp_type == 'Average of values':
  imputer2 = SimpleImputer(strategy ='mean')
  df['sepal_width']= imputer2.fit_transform(df[['sepal_width']])
if sepal_width_imp_type == 'custom':
  imputer2 = SimpleImputer(strategy ='constant',fill_value = sepal_width_imp_val)
  df['sepal_width']= imputer2.fit_transform(df[['sepal_width']])
if sepal_width_imp_type == 'median':
  imputer2 = SimpleImputer(strategy ='median')
  df['sepal_width']= imputer2.fit_transform(df[['sepal_width']])


if petal_length_imp_type == 'Average of values':
  imputer3 = SimpleImputer(strategy = 'mean')
  df['petal_length']= imputer3.fit_transform(df[['petal_length']])
if petal_length_imp_type == 'custom':
  imputer3 = SimpleImputer(strategy = 'constant',fill_value = petal_length_imp_val)
  df['petal_length']= imputer3.fit_transform(df[['petal_length']])
if petal_length_imp_type == 'median':
  imputer3 = SimpleImputer(strategy = 'median')
  df['petal_length']= imputer3.fit_transform(df[['petal_length']])


if petal_width_imp_type == 'Average of values':
  imputer4 = SimpleImputer(strategy = 'mean')
  df['petal_width']= imputer4.fit_transform(df[['petal_width']])
if petal_width_imp_type == 'custom':
  imputer4 = SimpleImputer(strategy = 'constant',fill_value = petal_width_imp_val)
  df['petal_width']= imputer4.fit_transform(df[['petal_width']])
if petal_width_imp_type == 'median':
  imputer4 = SimpleImputer(strategy = 'median')
  df['petal_width']= imputer4.fit_transform(df[['petal_width']])


from sklearn.preprocessing import LabelEncoder
if target_var != "species":
  df['species'] = LabelEncoder().fit_transform(df['species'])
   


In [8]:
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor

In [9]:
X = df.drop(columns=[target_var],axis=1)
y = df[target_var]

In [10]:
from numpy.ma.extras import column_stack
#feature reduction details
reduction_method = data["design_state_data"]["feature_reduction"]["feature_reduction_method"]

num = int(data["design_state_data"]["feature_reduction"]["num_of_features_to_keep"])

if reduction_method == 'No Reduction':
  feature_keep = num
  df_reduced = df.iloc[:,0:num+1]

if reduction_method == 'Correlation with target':
  feature_keep = num
  df_reduced = df.iloc[:,0:num+1]

if reduction_method == "Tree-based":
  feature_keep = num
  tree_num = int(data["design_state_data"]["feature_reduction"]["num_of_trees"])
  tree_depth = int(data["design_state_data"]["feature_reduction"]["depth_of_trees"])
  if prediction_type == 'Regression':
    rf = RandomForestRegressor(n_estimators = tree_num)
    rf.fit(X,y)
    rf.feature_importances_
    df_rf = pd.DataFrame({'features':X.columns,'importances':rf.feature_importances_})
    df_rf = df_rf.sort_values(by='importances',ascending = False)
    columns=[]
    for i in range(num):
      columns.append(df_rf.iloc[i,0])
    df_reduced = df.loc[:,columns]
    df_reduced = df_reduced.join(df[target_var])
  if prediction_type=="Classification":
    rf = RandomForestClassifier(n_estimators = tree_num)
    rf.fit(X,y)
    rf.feature_importances_
    df_rf = pd.DataFrame({'features':X.columns,'importances':rf.feature_importances_})
    df_rf = df_rf.sort_values(by='importances',ascending = False)
    columns=[]
    for i in range(num):
      columns.append(df_rf.iloc[i,0])
    df_reduced = df.loc[:,columns]
    df_reduced = df_reduced.join(df[target_var])


if reduction_method == "Principal Component Analysis":
  feature_keep == num
  pca = PCA(n_components = feature_keep)
  df_reduced = pca.fit_transform(df)

In [11]:
df_reduced.head()

,species,petal_length,sepal_width,sepal_length,petal_width
0,0,1.4,3.5,5.1,0.2
1,0,1.4,3.0,4.9,0.2
2,0,1.3,3.2,4.7,0.2
3,0,1.5,3.1,4.6,0.2
4,0,1.4,3.6,5.0,0.2


In [12]:
#Splitting the dataset
X = df_reduced.drop(columns=[target_var],axis=1)
y = df_reduced[target_var]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0)

In [14]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable


In [15]:
# import algorithms from sklearn
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor,GradientBoostingClassifier,GradientBoostingRegressor,ExtraTreesRegressor,ExtraTreesClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression,Ridge,Lasso,ElasticNet,SGDRegressor,SGDClassifier
from xgboost import XGBRegressor,XGBClassifier
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.svm import SVR,SVC
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
from sklearn.neural_network import MLPRegressor,MLPClassifier

In [16]:
from sklearn.model_selection import GridSearchCV
folds = data["design_state_data"]["hyperparameters"]["num_of_folds"]


In [17]:
if prediction_type =='Regression':
    e1 = data["design_state_data"]["algorithms"]["RandomForestRegressor"]["min_trees"]
    e2 = data["design_state_data"]["algorithms"]["RandomForestRegressor"]["max_trees"]
    d1 = data["design_state_data"]["algorithms"]["RandomForestRegressor"]["min_depth"]
    d2 = data["design_state_data"]["algorithms"]["RandomForestRegressor"]["max_depth"]
    s1 = data["design_state_data"]["algorithms"]["RandomForestRegressor"]["min_samples_per_leaf_min_value"]
    s2 = data["design_state_data"]["algorithms"]["RandomForestRegressor"]["min_samples_per_leaf_max_value"]

    params ={'n_estimators':[e1,e2],'max_depth':[d1,d2],"min_samples_leaf":[s1,s2]}
    g_rfr = GridSearchCV(RandomForestRegressor(),params,cv = folds)
    g_rfr.fit(X_train,y_train)
    dic_rfr = g_rfr.best_params_
    rfr = RandomForestRegressor(max_depth = dic_rfr['max_depth'],min_samples_leaf = dic_rfr['min_samples_leaf'],n_estimators = dic_rfr['n_estimators'])
    modelrfr = rfr.fit(X_train,y_train)
    y_prfr = rfr.predict(X_test)
    s_rfr1 = rfr.score(X_test,y_test)
    print(s_rfr1)

0.9214139256426638


In [18]:
if prediction_type =='Regression':
    #GBR
    e1 = data["design_state_data"]["algorithms"]["GBTRegressor"]["num_of_BoostingStages"]
    s1 = data["design_state_data"]["algorithms"]["GBTRegressor"]["min_subsample"]
    s2 = data["design_state_data"]["algorithms"]["GBTRegressor"]["max_subsample"]
    d1 = data["design_state_data"]["algorithms"]["GBTRegressor"]["min_depth"]
    d2 = data["design_state_data"]["algorithms"]["GBTRegressor"]["max_depth"]
    i1 = data["design_state_data"]["algorithms"]["GBTRegressor"]["min_iter"]
    i2 = data["design_state_data"]["algorithms"]["GBTRegressor"]["max_iter"]
    params = {'n_estimators':e1,'max_depth':[d1,d2],'n_iter_no_change':[i1,i2]}
    g_gbr = GridSearchCV(GradientBoostingRegressor(),params,cv = folds)
    g_gbr.fit(X_train,y_train);
    dic_gbr = g_gbr.best_params_
    gbr = GradientBoostingRegressor(max_depth = dic_gbr['max_depth'],n_estimators = dic_gbr['n_estimators'],n_iter_no_change = dic_gbr['n_iter_no_change'])
    modelgbr = gbr.fit(X_train,y_train)
    y_pgbr = gbr.predict(X_test)
    s_gbr= gbr.score(X_test,y_test)
    print(s_gbr)

0.9083120997272067


In [19]:
if prediction_type =='Regression':
    #Extratreeregressor
    e1 = data["design_state_data"]["algorithms"]["extra_random_trees"]['num_of_trees']
    d1 = data["design_state_data"]["algorithms"]["extra_random_trees"]['max_depth']
    l1 = data["design_state_data"]["algorithms"]["extra_random_trees"]['min_samples_per_leaf']
    params = {'n_estimators':e1,'max_depth':d1,'min_samples_leaf': l1}
    g_ext = GridSearchCV(ExtraTreesRegressor(),params,cv = folds)
    g_ext.fit(X_train,y_train);
    dic_ext = g_ext.best_params_
    dic_ext
    ext = ExtraTreesRegressor(max_depth = dic_ext['max_depth'],n_estimators = dic_ext['n_estimators'],min_samples_leaf=dic_ext['min_samples_leaf'])
    modelext = ext.fit(X_train,y_train)
    y_pext = ext.predict(X_test)
    s_ext = ext.score(X_test,y_test)
    print(s_ext)

-0.017871849836534226


In [20]:
if prediction_type =='Regression':
    #Linearregression
    lin=LinearRegression()
    modellin = lin.fit(X_train,y_train)
    y_plin = lin.predict(X_test)
    s_lin = lin.score(X_test,y_test)
    print(s_lin)


0.9323483696029023


In [22]:
if prediction_type =='Regression':
    #ridge
    i1=data["design_state_data"]["algorithms"]["RidgeRegression"]['min_iter']
    i2=data["design_state_data"]["algorithms"]["RidgeRegression"]['max_iter']
    a1 =data["design_state_data"]["algorithms"]["RidgeRegression"]['min_regparam']
    a2 =data["design_state_data"]["algorithms"]["RidgeRegression"]['max_regparam']
    params= {'max_iter':[i1,i2],'alpha':[a1,a2]}
    g_rid = GridSearchCV(Ridge(),params,cv = folds)
    g_rid.fit(X_train,y_train);
    dic_rid = g_rid.best_params_
    dic_rid
    rid = Ridge(max_iter = dic_rid['max_iter'],alpha = dic_rid['alpha'])
    modelrid = rid.fit(X_train,y_train)
    y_prid = rid.predict(X_test)
    s_rid = rid.score(X_test,y_test)
    print(s_rid)

0.9316430518164385


In [24]:
if prediction_type =='Regression':
    #lasso
    i1=data["design_state_data"]["algorithms"]["LassoRegression"]['min_iter']
    i2=data["design_state_data"]["algorithms"]["LassoRegression"]['max_iter']
    a1 =data["design_state_data"]["algorithms"]["LassoRegression"]['min_regparam']
    a2 =data["design_state_data"]["algorithms"]["LassoRegression"]['max_regparam']
    params= {'max_iter':[i1,i2],'alpha':[a1,a2]}
    g_las = GridSearchCV(Lasso(),params,cv = folds)
    g_las.fit(X_train,y_train);
    dic_las = g_las.best_params_
    dic_las
    las = Lasso(max_iter = dic_las['max_iter'],alpha = dic_las['alpha'])
    modellas = las.fit(X_train,y_train)
    y_plas = las.predict(X_test)
    s_las = las.score(X_test,y_test)
    print(s_las)

0.7871637749252313


In [25]:
if prediction_type =='Regression':
    #elasticnet
    i1=data["design_state_data"]["algorithms"]["ElasticNetRegression"]['min_iter']
    i2=data["design_state_data"]["algorithms"]["ElasticNetRegression"]['max_iter']
    a1 =data["design_state_data"]["algorithms"]["ElasticNetRegression"]['min_regparam']
    a2 =data["design_state_data"]["algorithms"]["ElasticNetRegression"]['max_regparam']
    e1 =data["design_state_data"]["algorithms"]["ElasticNetRegression"]['min_elasticnet']
    e2 =data["design_state_data"]["algorithms"]["ElasticNetRegression"]['max_elasticnet']
    params= {'max_iter':[i1,i2],'alpha':[a1,a2],'l1_ratio':[e1,e2]}
    g_els = GridSearchCV(ElasticNet(),params,cv = folds)
    g_els.fit(X_train,y_train);
    dic_els = g_els.best_params_
    dic_els
    els = ElasticNet(max_iter = dic_els['max_iter'],alpha = dic_els['alpha'],l1_ratio = dic_els['l1_ratio'])
    modelels = els.fit(X_train,y_train)
    y_pels = els.predict(X_test)
    s_els = els.score(X_test,y_test)
    print(s_els)

0.8536585607724704


In [26]:
if prediction_type =='Regression':
    #SGD
    t1=data["design_state_data"]["algorithms"]["SGD"]["tolerance"]
    if data["design_state_data"]["algorithms"]["SGD"]["use_elastic_net_regularization"]==True:
      e1 = 'elasticnet'
    else:
      e1 = 'l2'
    a1 = data["design_state_data"]["algorithms"]["SGD"]["alpha_value"]
    params = {'tol':[t1],'penalty':[e1],'alpha':a1}
    g_sgd = GridSearchCV(SGDRegressor(),params,cv = folds)
    g_sgd.fit(X_train,y_train);
    dic_sgd = g_sgd.best_params_
    dic_sgd
    sgd = SGDRegressor(tol=dic_sgd['tol'],penalty=dic_sgd['penalty'],alpha=dic_sgd['alpha'])
    modelsgd = sgd.fit(X_train,y_train)
    y_psgd = sgd.predict(X_test)
    s_sgd = sgd.score(X_test,y_test)
    print(s_sgd)

-0.029156691223380937


In [27]:
if prediction_type=="Regression":
    #XGbRegressor
    l1=data["design_state_data"]["algorithms"]["xg_boost"]["learningRate"]
    d1=data["design_state_data"]["algorithms"]["xg_boost"]["max_depth_of_tree"]
    params={'max_depth':d1,'learning_rate':l1}
    g_xgb = GridSearchCV(XGBRegressor(),params,cv = folds)
    g_xgb.fit(X_train,y_train);
    dic_xgb = g_xgb.best_params_
    dic_xgb
    xgb = XGBRegressor(max_depth=dic_xgb['max_depth'],learning_rate=dic_xgb['learning_rate'])
    modelxgb = xgb.fit(X_train,y_train)
    y_pxgb = xgb.predict(X_test);
    s_xgb = xgb.score(X_test,y_test)
    print(s_xgb)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py", line 706, in score
    return r2_score(y, y_pred, sample_weight=sample_weight)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_regression.py", line 789, in r2_score
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_regression.py", line 96, in _check_reg_targets
    y_pred =

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [28]:
if prediction_type=="Regression":
    #DTR
    d1 = data["design_state_data"]["algorithms"]["DecisionTreeRegressor"]["min_depth"]
    d2 = data["design_state_data"]["algorithms"]["DecisionTreeRegressor"]["max_depth"]
    s1 = data["design_state_data"]["algorithms"]["DecisionTreeRegressor"]["min_samples_per_leaf"]
    params = {'max_depth':[d1,d2],'min_samples_leaf':s1}
    g_dtr = GridSearchCV(DecisionTreeRegressor(),params,cv = folds)
    g_dtr.fit(X_train,y_train);
    dic_dtr = g_dtr.best_params_
    dic_dtr
    dtr = DecisionTreeRegressor(max_depth=dic_dtr['max_depth'],min_samples_leaf= dic_dtr['min_samples_leaf'])
    modeldtr = dtr.fit(X_train,y_train)
    y_pdtr = dtr.predict(X_test)
    s_dtr = dtr.score(X_test,y_test)
    print(s_dtr)

0.9038694958386728


In [29]:
if prediction_type=="Regression":
    #SVR
    c1 = data["design_state_data"]["algorithms"]["SVM"]['c_value']
    t1 = data["design_state_data"]["algorithms"]["SVM"]['tolerance']
    i1 = data["design_state_data"]["algorithms"]["SVM"]['max_iterations']
    params = {"C":c1,'tol':[t1],'max_iter':[i1]}
    g_svr = GridSearchCV(SVR(),params,cv = folds)
    g_svr.fit(X_train,y_train);
    dic_svr = g_svr.best_params_
    dic_svr
    svr =SVR(C=dic_svr["C"],tol = dic_svr['tol'],max_iter = dic_svr['max_iter'])
    modelsvr = svr.fit(X_train,y_train)
    y_psvr = svr.predict(X_test)
    s_svr = svr.score(X_test,y_test)
    print(s_svr)

-0.05997586769820429


In [30]:
if prediction_type=="Regression":
    #knn
    alg_name= data["design_state_data"]["algorithms"]["KNN"]["neighbour_finding_algorithm"]
    if alg_name=="Automatic":
      Algorithm = "auto"
    knn = KNeighborsRegressor(algorithm = Algorithm)
    modelknn = knn.fit(X_train,y_train)
    y_pknn = knn.predict(X_test)
    s_knn= knn.score(X_test,y_test)
    print(s_knn)

0.9356786145219675


In [33]:
if prediction_type=="Regression":
    #neural
    hl = data["design_state_data"]["algorithms"]["neural_network"]['hidden_layer_sizes']
    s1 = data["design_state_data"]["algorithms"]["neural_network"]['solver']
    if s1=='ADAM':
      sol = 'adam'
    b1 = data["design_state_data"]["algorithms"]["neural_network"]['beta_1']
    b2 = data["design_state_data"]["algorithms"]["neural_network"]['beta_2']
    ep = data["design_state_data"]["algorithms"]["neural_network"]['epsilon']
    es = data["design_state_data"]["algorithms"]["neural_network"]['early_stopping']
    m1 = data["design_state_data"]["algorithms"]["neural_network"]['momentum']
    params = {'hidden_layer_sizes':hl,"solver":[sol]}
    g_mlpr = GridSearchCV(MLPRegressor(),params,cv = folds)
    g_mlpr.fit(X_train,y_train)
    dic_mlpr = g_mlpr.best_params_
    dic_mlpr
    mlpr = MLPRegressor(solver=dic_mlpr['solver'],beta_1 = b1,beta_2 = b2 ,early_stopping= es, momentum = m1,hidden_layer_sizes=dic_mlpr['hidden_layer_sizes'])
    modelmlpr = mlpr.fit(X_train,y_train)
    y_pmlpr = mlpr.predict(X_test)
    s_mlpr= mlpr.score(X_test,y_test)
    print(s_mlpr)

0.8610019093348302


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.

In [35]:
#if prediction_type=="Classification":
    